<a href="https://colab.research.google.com/github/Neulvo/py_yum/blob/master/Dacon_jjcard_Final_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost

In [ ]:
from sklearn.impute import KNNImputer
from tqdm.notebook import tqdm
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import seaborn as sns
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
class serial_impute():
  
  def __init__(self,data):
    self.data=data
    self.amt=data[['REG_YYMM', 'STD_CLSS_NM','CARD_SIDO_NM','CSTMR_CNT','AMT','CNT']].groupby(['REG_YYMM', 'STD_CLSS_NM','CARD_SIDO_NM'],as_index=False).sum()
    self.sido_list = list(set(data['CARD_SIDO_NM']))
    self.std_list=list(set(data['STD_CLSS_NM']))
    self.month_list = list(set(data['REG_YYMM']))

#데이터 프레임 조작해서 원하는 형식으로 맞춰주기
  def pd_maker(self,sido,std):
    p_amt=self.amt[(self.amt['CARD_SIDO_NM']==sido)&(self.amt['STD_CLSS_NM']==std)]
    p_val=p_amt[['REG_YYMM','CSTMR_CNT','AMT']]
    p_val.replace(0,np.nan,inplace=True)
    p_val['MM']=p_val['REG_YYMM']%100
    p_val['YY']=p_val['REG_YYMM']//100
    p_val['MM']=p_val['MM'].astype(int)
    p_val['YY']=p_val['YY'].astype(int)
    amt_final=p_val[['YY','MM',"CSTMR_CNT",'AMT']]
    amt_final.reset_index(inplace=True)
    return amt_final

#비식별화 데이터 채워넣기 by sum(AMT) data
#원 데이터에서 submission 파일과 겹치는 칼럼 외의 칼럼들을 제거한 후 AMT를 더해주니
#도시, 업종별로 값이 존재하지 않는 데이터가 존재.
#비식별화 조치로 없어진 데이터는 추후에도 없어질 가능성이 높다 판단되 결측치 처리를 해줌.
#또는 추후에 생겨날 수 있는 데이터에 대해서도 고려해줌.
  def full_imputer(self):
    result_pd=pd.DataFrame(columns=['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT'])
    for sinan in tqdm(self.sido_list):
      for stnan in tqdm(self.std_list):
        sp_data=self.data[(self.data['CARD_SIDO_NM']==sinan)&(self.data['STD_CLSS_NM']==stnan)]    
        #위에서 만든 형태 지정 메소드 사용.
        amt_pd=self.pd_maker(sinan,stnan)
        #상관계수.
        d_corr=sp_data.corr()
        ca_corr=d_corr['CSTMR_CNT']['AMT']
        #값이 결측된 행이 10개 미만인 데이터에서만 impute 실행.
        if 0<amt_pd['AMT'].isnull().sum() <10:
        #상관계수가 0.66 이상인 데이터에 한해서 KNNImputer를 사용해 예측 실행.
        #CSTMR_CNT와 AMT의 전반적인 상관계수가 높기 때문 + 상관계수가 높아 예측값의 정확도도 높을 것이라 보기 때문.        
          if ca_corr>0.66:
            imputer = KNNImputer(n_neighbors=3, weights="uniform")
            nn_pd=pd.DataFrame(imputer.fit_transform(amt_pd))
            nn_pd.columns=amt_pd.columns
            nn_pd['CARD_SIDO_NM']=str(sinan)
            nn_pd['STD_CLSS_NM']=str(stnan)
            result_pd=pd.concat([result_pd,nn_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]])
          else:
        #울산 면세점, 전북 면세점 같은 경우는 값이 치우쳐짐 + 최근 데이터 결측이 심해 0으로 채워줌.
            if ((sinan=='울산')&(stnan=='면세점'))|((sinan=='전북')&(stnan=='내항 여객 운송업')):
              amt_pd.replace(np.nan,0,inplace=True)
              amt_pd['CARD_SIDO_NM']=str(sinan)
              amt_pd['STD_CLSS_NM']=str(stnan)
              result_pd=pd.concat([result_pd,amt_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]])
            else:
        #그 외의 데이터는 Iterativeimputer의 BayesianRidge를 활용해 결측치 처리.
        #결측이 있어도 나타날 가능성이 있다 판단되는 데이터들.
        #median 값을 사용.
              cc_pd=amt_pd[['YY','MM','AMT']]
              nn_pd = pd.DataFrame(IterativeImputer(random_state=8,initial_strategy='median',estimator=BayesianRidge()).fit_transform(cc_pd.values))
              nn_pd.columns=cc_pd.columns
              nn_pd['CARD_SIDO_NM']=str(sinan)
              nn_pd['STD_CLSS_NM']=str(stnan)
              result_pd=pd.concat([result_pd,nn_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]])
        #결측된 행이 10개 이상이면 비식별화 확률이 높다 판단하여 모두 0으로 채워주었다.
        else:
          amt_pd.replace(np.nan,0,inplace=True)
          amt_pd['CARD_SIDO_NM']=str(sinan)
          amt_pd['STD_CLSS_NM']=str(stnan)
          result_pd=pd.concat([result_pd,amt_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]])
    result_pd=result_pd.reset_index()
  
    return result_pd[['YY','MM','CARD_SIDO_NM','STD_CLSS_NM','AMT']]

In [ ]:
#원본 데이터(201901-202003) 가져오기
data_jj=pd.read_csv('/content/drive/My Drive/data/dacon_data/201901-202003.csv')
#submission 가져오기
sub=pd.read_csv('/content/drive/My Drive/data/dacon_data/submission.csv')

In [ ]:
#04 월 데이터
data_04=pd.read_csv('/content/drive/My Drive/data/dacon_data/202004.csv')

In [ ]:
#비식별 데이터를 처리하기 위해 submission 기준으로 필요한 칼럼들을 뽑아옴.

sido_list = list(set(data_jj['CARD_SIDO_NM']))
std_list=list(set(data_jj['STD_CLSS_NM']))
month_list = list(set(data_jj['REG_YYMM']))

In [ ]:
# NaN 데이터 생성. 0또는 결측값 채워주기 위한 용도

data_nan_n =[]
for month in month_list:
  for sido in sido_list:
    for std in std_list:
      data_nans=[int(month),sido,std,np.nan,np.nan,np.nan]
      data_nan_n.append(data_nans)
data_n=pd.DataFrame(data_nan_n,
                    columns=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM','CSTMR_CNT','AMT','CNT'])
data_jj_n=pd.concat([data_jj,data_n],ignore_index=True)

In [ ]:
# 비식별화 데이터 처리.
full_nn=serial_impute(data_jj_n).full_imputer()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

...

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4172: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [ ]:
# 시도, 업종별 AMT 값의 분포를 기준으로 군집처리한 데이터
na_base=pd.read_csv('/content/drive/My Drive/Dacon/variable/na_based_var.csv',encoding='EUC-KR')
na_b=na_base[['CARD_SIDO_NM','STD_CLSS_NM','cluster']]
na_dp=na_b.drop_duplicates()

In [ ]:
# 4월에 비식별화된 데이터가 많다 판단해 3월까지의 데이터를 사용해 학습을 시켜주기로 결정.
full_n3=full_nn.merge(na_dp,how='inner',on=['CARD_SIDO_NM','STD_CLSS_NM'])

In [ ]:
# predict(제출) 데이터의 형식도 맞춰줌. 
sub_nab=sub.merge(na_dp,how='inner',on=['CARD_SIDO_NM','STD_CLSS_NM'])
sub_nab.sort_values(by='id',inplace=True)

In [ ]:
class x_y_maker():
  def __init__(self,data):
    self.data=data
    #데이터 카피
    self.ori_data=data.copy()
    #AGE->AGE_GROUP
    #시도간 이동 라벨링
    self.encoder_s=LabelEncoder()
    self.encoder_c=LabelEncoder() 
    self.encoder_cls=LabelEncoder()
    self.stscaler=StandardScaler()

#특성 더 넣고 싶은 게 있으면 여기서 칼럼 만들어주면 된다.
#잘 모르겠으면 그냥 물어봐줘...
  
  def col_setting(self,full_data):
    #STD_CLSS_LB 만들기 (업종 데이터 라벨화)
    sample_label_s=self.encoder_s.transform(full_data['STD_CLSS_NM'])
    full_data['STD_CLSS_LB']=sample_label_s
    #CARD_SIDO_LB 만들기 (시도 데이터 라벨화)
    sample_label_c=self.encoder_c.transform(full_data['CARD_SIDO_NM'])
    full_data['CARD_SIDO_LB']=sample_label_c
    #cluster 라벨화
    sample_label_cls=self.encoder_cls.transform(full_data['cluster'])
    full_data['CLUS_LB']=sample_label_cls
    # MM, YY 나눠주기
    #필요한 칼럼만 축소
    actual_data=full_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB','CLUS_LB','AMT']]
    
    return actual_data 

    # 원데이터 변형
  def data_maker(self):
    # CARD_CCG_NM, HOM_SIDO_NM, HOM_CCG_NM, SEX_CTGO, AGE, AGE_GROUP, CSTMR_CNT, CNT 삭제
    # CSTMR_CNT, CNT를 SUB에 반영할 수가 없다. ㅠ
    # 원데이터에서는 따로 NaN을 집어넣어주지 않음. NaN 때문에 학습이 안되서.
    # 만약 결측치 처리한 게 있으면 원데이터 대신 넣어주면 된다.
    mak_data=self.ori_data[['YY','MM', 'CARD_SIDO_NM', 'STD_CLSS_NM','cluster',
                            'AMT']].groupby(['YY','MM', 'CARD_SIDO_NM', 'STD_CLSS_NM','cluster']).agg(['sum']).reset_index()
    mak_data.columns=['YY','MM', 'CARD_SIDO_NM', 'STD_CLSS_NM','cluster','AMT']
    #라벨 인코딩
    self.encoder_s.fit(mak_data['STD_CLSS_NM'])
    self.encoder_c.fit(mak_data['CARD_SIDO_NM'])
    self.encoder_cls.fit(mak_data['cluster'])
    t_data=self.col_setting(mak_data)
    X_t=t_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB','CLUS_LB']]
    y_ft=t_data[['AMT']]
    #기울어진 데이터 => log변환 후 StandardScaling
    y_ts=self.stscaler.fit_transform(np.log1p(y_ft))
    #shape 맞춰주기
    y_t=y_ts.flatten()
    return X_t, y_t

    # sub데이터 변형
  def sub_maker(self,sub):
    sub_m=sub.copy()
    sub_m['YY']=sub_m['REG_YYMM']//100
    sub_m['MM']=sub_m['REG_YYMM']%100
    sub_data=self.col_setting(sub_m)
    X_s=sub_data[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB','CLUS_LB']]
    y_fs=sub_data[['AMT']]
    #shape 맞춰주기
    y_s=y_fs.values.flatten()
    return X_s, y_s

    # sub데이터 디코드, sub_t에는 서브미션에서 AMT제외한 거, sub_p에는 predict value 넣어주면 됨.
  def sub_decoder(self,sub_t,sub_p):
    sub_d=sub_t.copy()
    sub_d=sub_d[['YY','MM', 'CARD_SIDO_LB', 'STD_CLSS_LB']]
    #CARD SIDO, STD CLSS 디코딩
    sub_d['STD_CLSS_NM']=self.encoder_s.inverse_transform(sub_d['STD_CLSS_LB'])
    sub_d['CARD_SIDO_NM']=self.encoder_c.inverse_transform(sub_d['CARD_SIDO_LB'])
    #연도 돌려주기
    sub_d['REG_YYMM']=sub_d['YY']*100+sub_d['MM']
    #Standard Scaling decoding
    st_p=self.stscaler.inverse_transform(sub_p)
    #log1p decoding
    fin_p= np.expm1(st_p)
    #소수값 반올림
    sub_d['AMT']=np.round(fin_p,0)
    #50미만 0으로 환원
    sub_d['AMT']=np.where(sub_d['AMT']<50,0,sub_d['AMT'])
    #최종 셋 맞춰주기
    sub_d=sub_d[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
    return sub_t

In [ ]:
#객체 생성
xy_maker = x_y_maker(full_n3)

In [ ]:
# 객체 내 메서드 실행 => 원 데이터 변형
x,y=xy_maker.data_maker()
# 객체 내 메서드 실행 => sub 데이터 변형
X_sub, y_sub=xy_maker.sub_maker(sub_nab)

In [ ]:
# 트레인, 테스트 셋 분리 random_state=777
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.3, random_state=777,shuffle = True)

In [ ]:
xgb_final=xgb.XGBRegressor(alpha=0.03,colsample_bytree=0.7,learning_rate=0.05,max_depth=10,
                      min_child_weight=2,n_estimators=2500,random_state=545,objective='reg:squarederror',
                     subsample=0.7,reg_lambda=0.01)

In [ ]:
xgb_final.fit(X_train,y_train)

XGBRegressor(alpha=0.03, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=10, min_child_weight=2, missing=None, n_estimators=2500,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=545, reg_alpha=0, reg_lambda=0.01, scale_pos_weight=1,
             seed=None, silent=None, subsample=0.7, verbosity=1)

In [ ]:
xgb_final.score(X_test,y_test)

0.9676084916530527

In [ ]:
xgb_final.fit(x,y,verbose=0)
submiss=xy_maker.sub_decoder(X_sub,xgb_final.predict(X_sub))

In [ ]:
submiss.index.name='id'
submiss.to_csv('/content/drive/My Drive/Dacon/final_submission/test.csv', encoding='utf-8-sig')